In [2]:
import pandas as pd
import numpy as np
import pickle
import glob
import os
import re
import kstar
from kstar import config, helpers, mapping, calculate

# phospho_data = pd.read_csv("proteomics\data\phospho_data\ptrc_ex18_crosstab_phospho_siteID_prot_corrected.txt", sep = "\t")


ModuleNotFoundError: No module named 'kstar'

Using sig sites as input

In [2]:
#define the directory where mapped dataset and run information will be saved.
odir = '.'
#if directory doesn't exist yet, create it
if not os.path.exists(f"{odir}/MAPPED_DATA"):
    os.mkdir(f"{odir}/MAPPED_DATA")

#Define the log name
logName = 'exp_18_cells_sigs'
#intialize logger
mapping_log = helpers.get_logger(f"mapping_{logName}", f"{odir}/MAPPED_DATA/mapping_{logName}.log")

mapDict = {'peptide':'Peptide_kstar', 'accession_id':'Protein_kstar'}
phospho_data = pd.read_csv("..\comparisons\\treatments_vs_parental\Tables\KSTAR_full_input_sigs.txt", sep = "\t")

exp_mapper = mapping.ExperimentMapper(experiment = phospho_data,
                                      columns = mapDict,
                                      logger = mapping_log)
#save mapped dataset
exp_mapper.experiment.to_csv(f"{odir}/MAPPED_DATA/{logName}_mapped.tsv", sep = '\t', index = False)

In [3]:
#Name
logName_new = logName + '_Y,ST'
#if directory does not exits, create it
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
#intialize log file
activity_log = helpers.get_logger(f"activity_{logName_new}", f"{odir}/RESULTS/activity_{logName_new}.log")

experiment = pd.read_csv(f'{odir}/MAPPED_DATA/{logName}_mapped.tsv', sep = '\t', index_col = 0)

#Load the pickles containing the 50 pruned networks for tyrosine kinases
phospho_types = ['ST'] #running on this type of kinase/substrate network
networks = {}
# networks['ST'] = pickle.load(open(".\PRUNED NETWORKS\PhosphoPICK\PhsphoPICK_pruning_ST_compendia_2000_limit_10.p", "rb"))
networks['ST'] = pickle.load(open(".\PRUNED NETWORKS\KSDB\KSDB_pruning_ST_30_limit_15.p", "rb"))

In [ ]:
#Checking counts 
agg = 'mean'
greater = False
threshold = -0.01
data_columns = None

#intialize KinaseActivity class object
kinact = calculate.KinaseActivity(experiment, activity_log,data_columns = data_columns, phospho_type=['S', 'T'])
#convert evidence into binary evidence based on the provided threshold
evidence_binary = kinact.create_binary_evidence(agg = agg, threshold = threshold,  greater = greater)
data_cols = [col for col in evidence_binary.columns if 'data:' in col]
evidence_binary[data_cols].sum()

In [4]:
agg = 'mean'
greater = False
threshold = -0.01
data_columns = None

#Create activity log: if already did this, ignore.
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
activity_log = helpers.get_logger(f"activity_{logName}", f"{odir}/RESULTS/activity_{logName}.log")

kinact_dict = calculate.enrichment_analysis(experiment, activity_log, networks, phospho_types = phospho_types,
                                                agg =agg, threshold = threshold,
                                                greater = greater, PROCESSES = 4)

#Set the number of random experiments
num_random_experiments = 150
#Generate random experiments
calculate.randomized_analysis(kinact_dict, activity_log, num_random_experiments, PROCESSES = 4)

calculate.Mann_Whitney_analysis(kinact_dict, activity_log, number_sig_trials = 100, PROCESSES = 4)

activities = kinact_dict['ST'].activities_mann_whitney.copy()
log_results = -np.log10(activities)
activities = activities[['data:Early_and_G_vs_Parental_and_none',
                         'data:Late_and_G_vs_Parental_and_none',
                         'data:Early_and_GD_vs_Parental_and_none',
                         'data:Late_and_GD_vs_Parental_and_none',
                         'data:Early_and_GV_vs_Parental_and_none',
                         'data:Late_and_GV_vs_Parental_and_none',
                         'data:Early_and_GVD_vs_Parental_and_none',
                         'data:Late_and_GVD_vs_Parental_and_none']]
fpr = kinact_dict['ST'].fpr_mann_whitney.copy()

for col in activities.columns:
    activities[col + "_fdr"] = fpr[col]

activities.to_csv("KSDB_KSTAR_ST_sigs_kinase_activity_negative.txt", sep = "\t")

c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:138: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(self.aggregate).reset_index()
c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:454: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(agg).reset_index()


In [5]:
agg = 'mean'
greater = True
threshold = 0.01
data_columns = None

#Create activity log: if already did this, ignore.
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
activity_log = helpers.get_logger(f"activity_{logName}", f"{odir}/RESULTS/activity_{logName}.log")

kinact_dict = calculate.enrichment_analysis(experiment, activity_log, networks, phospho_types = phospho_types,
                                                agg =agg, threshold = threshold,
                                                greater = greater, PROCESSES = 4)

#Set the number of random experiments
num_random_experiments = 150
#Generate random experiments
calculate.randomized_analysis(kinact_dict, activity_log, num_random_experiments, PROCESSES = 4)

calculate.Mann_Whitney_analysis(kinact_dict, activity_log, number_sig_trials = 100, PROCESSES = 4)

activities = kinact_dict['ST'].activities_mann_whitney.copy()
log_results = -np.log10(activities)
activities = activities[['data:Early_and_G_vs_Parental_and_none',
                         'data:Late_and_G_vs_Parental_and_none',
                         'data:Early_and_GD_vs_Parental_and_none',
                         'data:Late_and_GD_vs_Parental_and_none',
                         'data:Early_and_GV_vs_Parental_and_none',
                         'data:Late_and_GV_vs_Parental_and_none',
                         'data:Early_and_GVD_vs_Parental_and_none',
                         'data:Late_and_GVD_vs_Parental_and_none']]
fpr = kinact_dict['ST'].fpr_mann_whitney.copy()

for col in activities.columns:
    activities[col + "_fdr"] = fpr[col]

activities.to_csv("KSDB_KSTAR_ST_sigs_kinase_activity_positive.txt", sep = "\t")

c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:138: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(self.aggregate).reset_index()
c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:454: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(agg).reset_index()


Using sig only, cutoff of 1.

In [6]:
agg = 'mean'
greater = False
threshold = -1
data_columns = None

#Create activity log: if already did this, ignore.
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
activity_log = helpers.get_logger(f"activity_{logName}", f"{odir}/RESULTS/activity_{logName}.log")

kinact_dict = calculate.enrichment_analysis(experiment, activity_log, networks, phospho_types = phospho_types,
                                                agg =agg, threshold = threshold,
                                                greater = greater, PROCESSES = 4)

#Set the number of random experiments
num_random_experiments = 150
#Generate random experiments
calculate.randomized_analysis(kinact_dict, activity_log, num_random_experiments, PROCESSES = 4)

calculate.Mann_Whitney_analysis(kinact_dict, activity_log, number_sig_trials = 100, PROCESSES = 4)

activities = kinact_dict['ST'].activities_mann_whitney.copy()
log_results = -np.log10(activities)
activities = activities[['data:Early_and_G_vs_Parental_and_none',
                         'data:Late_and_G_vs_Parental_and_none',
                         'data:Early_and_GD_vs_Parental_and_none',
                         'data:Late_and_GD_vs_Parental_and_none',
                         'data:Early_and_GV_vs_Parental_and_none',
                         'data:Late_and_GV_vs_Parental_and_none',
                         'data:Early_and_GVD_vs_Parental_and_none',
                         'data:Late_and_GVD_vs_Parental_and_none']]
fpr = kinact_dict['ST'].fpr_mann_whitney.copy()

for col in activities.columns:
    activities[col + "_fdr"] = fpr[col]

activities.to_csv("KSDB_KSTAR_ST_sigs_kinase_activity_negative1.txt", sep = "\t")

c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:138: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(self.aggregate).reset_index()
c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:454: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(agg).reset_index()
c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log10
  re

In [7]:
agg = 'mean'
greater = True
threshold = 1
data_columns = None

#Create activity log: if already did this, ignore.
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
activity_log = helpers.get_logger(f"activity_{logName}", f"{odir}/RESULTS/activity_{logName}.log")

kinact_dict = calculate.enrichment_analysis(experiment, activity_log, networks, phospho_types = phospho_types,
                                                agg =agg, threshold = threshold,
                                                greater = greater, PROCESSES = 4)

#Set the number of random experiments
num_random_experiments = 150
#Generate random experiments
calculate.randomized_analysis(kinact_dict, activity_log, num_random_experiments, PROCESSES = 4)

calculate.Mann_Whitney_analysis(kinact_dict, activity_log, number_sig_trials = 100, PROCESSES = 4)

activities = kinact_dict['ST'].activities_mann_whitney.copy()
log_results = -np.log10(activities)
activities = activities[['data:Early_and_G_vs_Parental_and_none',
                         'data:Late_and_G_vs_Parental_and_none',
                         'data:Early_and_GD_vs_Parental_and_none',
                         'data:Late_and_GD_vs_Parental_and_none',
                         'data:Early_and_GV_vs_Parental_and_none',
                         'data:Late_and_GV_vs_Parental_and_none',
                         'data:Early_and_GVD_vs_Parental_and_none',
                         'data:Late_and_GVD_vs_Parental_and_none']]
fpr = kinact_dict['ST'].fpr_mann_whitney.copy()

for col in activities.columns:
    activities[col + "_fdr"] = fpr[col]

activities.to_csv("KSDB_KSTAR_ST_sigs_kinase_activity_positive1.txt", sep = "\t")

c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:138: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(self.aggregate).reset_index()
c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:454: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(agg).reset_index()
c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log10
  re

Using full input, not just significant ones. Cutoff of 1, with PhosphoPICK networks

In [2]:
#define the directory where mapped dataset and run information will be saved.
odir = '.'
#if directory doesn't exist yet, create it
if not os.path.exists(f"{odir}/MAPPED_DATA"):
    os.mkdir(f"{odir}/MAPPED_DATA")

#Define the log name
logName = 'exp_18_cells_full'
#intialize logger
mapping_log = helpers.get_logger(f"mapping_{logName}", f"{odir}/MAPPED_DATA/mapping_{logName}.log")

mapDict = {'peptide':'Peptide_kstar', 'accession_id':'Protein_kstar'}
phospho_data = pd.read_csv("..\comparisons\\treatments_vs_parental\Tables\KSTAR_full_input.txt", sep = "\t")

exp_mapper = mapping.ExperimentMapper(experiment = phospho_data,
                                      columns = mapDict,
                                      logger = mapping_log)
#save mapped dataset
exp_mapper.experiment.to_csv(f"{odir}/MAPPED_DATA/{logName}_mapped.tsv", sep = '\t', index = False)

In [4]:
#Name
logName_new = logName + '_Y,ST'
#if directory does not exits, create it
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
#intialize log file
activity_log = helpers.get_logger(f"activity_{logName_new}", f"{odir}/RESULTS/activity_{logName_new}.log")

experiment = pd.read_csv(f'{odir}/MAPPED_DATA/{logName}_mapped.tsv', sep = '\t', index_col = 0)

#Load the pickles containing the 50 pruned networks for tyrosine kinases
phospho_types = ['ST'] #running on this type of kinase/substrate network
networks = {}
networks['ST'] = pickle.load(open(".\PRUNED NETWORKS\PhosphoPICK\PhsphoPICK_pruning_ST_compendia_2000_limit_10.p", "rb"))
# networks['ST'] = pickle.load(open(".\PRUNED NETWORKS\KSDB\KSDB_pruning_ST_30_limit_15.p", "rb"))

In [5]:
agg = 'mean'
greater = False
threshold = -1
data_columns = None

#intialize KinaseActivity class object
## TYPO HERE!! MUST USE 'S', 'T', NOT 'ST'.
# kinact = calculate.KinaseActivity(experiment, activity_log, data_columns = data_columns, phospho_type=['Y', 'S', 'T'])   ## TYPO HERE!! MUST USE 'S', 'T', NOT 'ST'.
## convert evidence into binary evidence based on the provided threshold
# evidence_binary = kinact.create_binary_evidence(agg = agg, threshold = threshold,  greater = greater)

# data_cols = [col for col in evidence_binary.columns if 'data:' in col]
# evidence_binary[data_cols].sum()


#Create activity log: if already did this, ignore.
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
activity_log = helpers.get_logger(f"activity_{logName}", f"{odir}/RESULTS/activity_{logName}.log")

kinact_dict = calculate.enrichment_analysis(experiment, activity_log, networks, phospho_types = phospho_types,
                                                agg =agg, threshold = threshold,
                                                greater = greater, PROCESSES = 4)

#Set the number of random experiments
num_random_experiments = 150
#Generate random experiments
calculate.randomized_analysis(kinact_dict, activity_log, num_random_experiments, PROCESSES = 4)

calculate.Mann_Whitney_analysis(kinact_dict, activity_log, number_sig_trials = 100, PROCESSES = 4)

activities = kinact_dict['ST'].activities_mann_whitney.copy()
log_results = -np.log10(activities)
activities = activities[['data:Early_and_G_vs_Parental_and_none',
                         'data:Late_and_G_vs_Parental_and_none',
                         'data:Early_and_GD_vs_Parental_and_none',
                         'data:Late_and_GD_vs_Parental_and_none',
                         'data:Early_and_GV_vs_Parental_and_none',
                         'data:Late_and_GV_vs_Parental_and_none',
                         'data:Early_and_GVD_vs_Parental_and_none',
                         'data:Late_and_GVD_vs_Parental_and_none']]
fpr = kinact_dict['ST'].fpr_mann_whitney.copy()

for col in activities.columns:
    activities[col + "_fdr"] = fpr[col]

activities.to_csv("PhosphoPICK_KSTAR_ST_all_kinase_activity_negative1.txt", sep = "\t")

c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:138: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(self.aggregate).reset_index()
c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:454: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(agg).reset_index()


In [6]:
agg = 'mean'
greater = True
threshold = 1
data_columns = None

#intialize KinaseActivity class object
## TYPO HERE!! MUST USE 'S', 'T', NOT 'ST'.
# kinact = calculate.KinaseActivity(experiment, activity_log, data_columns = data_columns, phospho_type=['Y', 'S', 'T'])   ## TYPO HERE!! MUST USE 'S', 'T', NOT 'ST'.
#convert evidence into binary evidence based on the provided threshold
# evidence_binary = kinact.create_binary_evidence(agg = agg, threshold = threshold,  greater = greater)

# data_cols = [col for col in evidence_binary.columns if 'data:' in col]
# evidence_binary[data_cols].sum()


#Create activity log: if already did this, ignore.
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
activity_log = helpers.get_logger(f"activity_{logName}", f"{odir}/RESULTS/activity_{logName}.log")

kinact_dict = calculate.enrichment_analysis(experiment, activity_log, networks, phospho_types = phospho_types,
                                                agg =agg, threshold = threshold,
                                                greater = greater, PROCESSES = 4)

#Set the number of random experiments
num_random_experiments = 150
#Generate random experiments
calculate.randomized_analysis(kinact_dict, activity_log, num_random_experiments, PROCESSES = 4)

calculate.Mann_Whitney_analysis(kinact_dict, activity_log, number_sig_trials = 100, PROCESSES = 4)

activities = kinact_dict['ST'].activities_mann_whitney.copy()
log_results = -np.log10(activities)
activities = activities[['data:Early_and_G_vs_Parental_and_none',
                         'data:Late_and_G_vs_Parental_and_none',
                         'data:Early_and_GD_vs_Parental_and_none',
                         'data:Late_and_GD_vs_Parental_and_none',
                         'data:Early_and_GV_vs_Parental_and_none',
                         'data:Late_and_GV_vs_Parental_and_none',
                         'data:Early_and_GVD_vs_Parental_and_none',
                         'data:Late_and_GVD_vs_Parental_and_none']]
fpr = kinact_dict['ST'].fpr_mann_whitney.copy()

for col in activities.columns:
    activities[col + "_fdr"] = fpr[col]

activities.to_csv("PhosphoPICK_KSTAR_ST_all_kinase_activity_positive1.txt", sep = "\t")

c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:138: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(self.aggregate).reset_index()
c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:454: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(agg).reset_index()


20 networks, kinase size 50. KDSB

In [1]:
import pandas as pd
import numpy as np
import pickle
import glob
import os
import re

from kstar import config, helpers, mapping, calculate

#define the directory where mapped dataset and run information will be saved.
odir = '.'
#if directory doesn't exist yet, create it
if not os.path.exists(f"{odir}/MAPPED_DATA"):
    os.mkdir(f"{odir}/MAPPED_DATA")

#Define the log name
logName = 'exp_18_cells_sigs'
#intialize logger
mapping_log = helpers.get_logger(f"mapping_{logName}", f"{odir}/MAPPED_DATA/mapping_{logName}.log")

mapDict = {'peptide':'Peptide_kstar', 'accession_id':'Protein_kstar'}
phospho_data = pd.read_csv("..\comparisons\\treatments_vs_parental\Tables\KSTAR_full_input_sigs.txt", sep = "\t")

exp_mapper = mapping.ExperimentMapper(experiment = phospho_data,
                                      columns = mapDict,
                                      logger = mapping_log)
#save mapped dataset
exp_mapper.experiment.to_csv(f"{odir}/MAPPED_DATA/{logName}_mapped.tsv", sep = '\t', index = False)

#Name
logName_new = logName + '_Y,ST'
#if directory does not exits, create it
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
#intialize log file
activity_log = helpers.get_logger(f"activity_{logName_new}", f"{odir}/RESULTS/activity_{logName_new}.log")

experiment = pd.read_csv(f'{odir}/MAPPED_DATA/{logName}_mapped.tsv', sep = '\t', index_col = 0)

#Load the pickles containing the 50 pruned networks for tyrosine kinases
phospho_types = ['ST'] #running on this type of kinase/substrate network
networks = {}
# networks['ST'] = pickle.load(open(".\PRUNED NETWORKS\PhosphoPICK\PhsphoPICK_pruning_ST_compendia_2000_limit_10.p", "rb"))
networks['ST'] = pickle.load(open(".\PRUNED NETWORKS\KSDB\KSDB_pruning_ST_50_limit_15.p", "rb"))

In [2]:
agg = 'mean'
greater = True
threshold = 1
data_columns = None

#Create activity log: if already did this, ignore.
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
activity_log = helpers.get_logger(f"activity_{logName}", f"{odir}/RESULTS/activity_{logName}.log")

kinact_dict = calculate.enrichment_analysis(experiment, activity_log, networks, phospho_types = phospho_types,
                                                agg =agg, threshold = threshold,
                                                greater = greater, PROCESSES = 4)

#Set the number of random experiments
num_random_experiments = 150
#Generate random experiments
calculate.randomized_analysis(kinact_dict, activity_log, num_random_experiments, PROCESSES = 4)

calculate.Mann_Whitney_analysis(kinact_dict, activity_log, number_sig_trials = 100, PROCESSES = 4)

activities = kinact_dict['ST'].activities_mann_whitney.copy()
log_results = -np.log10(activities)
activities = activities[['data:Early_and_G_vs_Parental_and_none',
                         'data:Late_and_G_vs_Parental_and_none',
                         'data:Early_and_GD_vs_Parental_and_none',
                         'data:Late_and_GD_vs_Parental_and_none',
                         'data:Early_and_GV_vs_Parental_and_none',
                         'data:Late_and_GV_vs_Parental_and_none',
                         'data:Early_and_GVD_vs_Parental_and_none',
                         'data:Late_and_GVD_vs_Parental_and_none']]
fpr = kinact_dict['ST'].fpr_mann_whitney.copy()

for col in activities.columns:
    activities[col + "_fdr"] = fpr[col]

activities.to_csv("KSDB_KSTAR_ST_20net_sigs_kinase_activity_positive1.txt", sep = "\t")

c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:138: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(self.aggregate).reset_index()
c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:454: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(agg).reset_index()


In [3]:
agg = 'mean'
greater = False
threshold = -1
data_columns = None

#Create activity log: if already did this, ignore.
if not os.path.exists(f"{odir}/RESULTS"):
    os.mkdir(f"{odir}/RESULTS")
activity_log = helpers.get_logger(f"activity_{logName}", f"{odir}/RESULTS/activity_{logName}.log")

kinact_dict = calculate.enrichment_analysis(experiment, activity_log, networks, phospho_types = phospho_types,
                                                agg =agg, threshold = threshold,
                                                greater = greater, PROCESSES = 4)

#Set the number of random experiments
num_random_experiments = 150
#Generate random experiments
calculate.randomized_analysis(kinact_dict, activity_log, num_random_experiments, PROCESSES = 4)

calculate.Mann_Whitney_analysis(kinact_dict, activity_log, number_sig_trials = 100, PROCESSES = 4)

activities = kinact_dict['ST'].activities_mann_whitney.copy()
log_results = -np.log10(activities)
activities = activities[['data:Early_and_G_vs_Parental_and_none',
                         'data:Late_and_G_vs_Parental_and_none',
                         'data:Early_and_GD_vs_Parental_and_none',
                         'data:Late_and_GD_vs_Parental_and_none',
                         'data:Early_and_GV_vs_Parental_and_none',
                         'data:Late_and_GV_vs_Parental_and_none',
                         'data:Early_and_GVD_vs_Parental_and_none',
                         'data:Late_and_GVD_vs_Parental_and_none']]
fpr = kinact_dict['ST'].fpr_mann_whitney.copy()

for col in activities.columns:
    activities[col + "_fdr"] = fpr[col]

activities.to_csv("KSDB_KSTAR_ST_20net_sigs_kinase_activity_negative1.txt", sep = "\t")

c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:138: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(self.aggregate).reset_index()
c:\Users\poss982\Anaconda3\envs\KSTAR\lib\site-packages\kstar\calculate.py:454: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evidence = self.evidence.groupby([config.KSTAR_ACCESSION, config.KSTAR_SITE]).agg(agg).reset_index()
